# Importing Requirements

In [2]:
import pandas as pd
import numpy as np

# Declaring Helper Functions

In [3]:
def adjusted_probs(home_odds, draw_odds, away_odds):

    total_probs = sum([(1/odds) * 100 for odds in [home_odds, draw_odds, away_odds]])

    return tuple(((1/odds) * 100 )/total_probs for odds in [home_odds, draw_odds, away_odds])

In [4]:
adjusted_probs(1.67, 4, 5)[0]

0.5709391949757351

In [5]:
def theil_score(adj_home, adj_draw, adj_away):

    return sum([i * np.log(1/i) for i in [adj_home, adj_draw, adj_away]])

In [6]:
theil_score(0.368604, 0.255565, 0.375831)

1.0843346815152448

# Loading Data

In [14]:
(

    pd.read_csv('../data/N1-2.csv')
    .filter(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A'])
    .rename(columns = {
        'B365H': 'odds_home', 
        'B365D': 'odds_draw', 
        'B365A': 'odds_away'
    })
    .assign(
        Date=lambda x: pd.to_datetime(x['Date'], format='%d/%m/%Y'),
        adj_home=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[0], axis=1),
        adj_draw=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[1], axis=1),
        adj_away=lambda x: x.apply(lambda row: adjusted_probs(row['odds_home'], row['odds_draw'], row['odds_away'])[2], axis=1),
        abs_diff_adj_odds=lambda x: abs(x.adj_home - x.adj_away),
        theil_score = lambda x: x.apply(lambda row: theil_score(row['adj_home'], row['adj_draw'], row['adj_away']), axis=1),
    )
    .query('Date >= "2023-12-01"') # selecting the latest round of games
    .sort_values('abs_diff_adj_odds', ascending=True)
    # .sort_values('theil_score', ascending=False)
    .head(5)
)

,Div,Date,HomeTeam,AwayTeam,odds_home,odds_draw,odds_away,adj_home,adj_draw,adj_away,abs_diff_adj_odds,theil_score
116,N1,2023-12-02,Volendam,Zwolle,2.60,3.75,2.55,0.368604,0.255565,0.375831,0.007228,1.084335
119,N1,2023-12-02,Heracles,Sparta Rotterdam,2.70,3.60,2.45,0.350626,0.262970,0.386404,0.035778,1.086142
120,N1,2023-12-03,Feyenoord,PSV Eindhoven,2.20,3.60,3.00,0.426540,0.260664,0.312796,0.113744,1.077434
118,N1,2023-12-02,Waalwijk,Excelsior,2.05,4.00,3.25,0.466577,0.239121,0.294302,0.172275,1.057793
117,N1,2023-12-02,For Sittard,Vitesse,2.00,3.60,3.60,0.473684,0.263158,0.263158,0.210526,1.056576
